In [ ]:
from itertools import starmap
from functools import partial
from collections import namedtuple
import random
import datetime

from IPython import display
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image, Label, HTML, Dropdown, IntProgress
import pandas
import jinja2
from pony import orm

import db
db.use_db('minitest1_data/minitest1.sqlite')

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 2,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

# Widget definitions

In [ ]:
### Views -- all the individual widgets and UI state

## Immutable app state
with orm.db_session:
    n_submissions = orm.max(sub.id for sub in db.Submission)
    n_problems = db.Problem.select().count()
    students = tuple(starmap('{} ({} {})'.format, orm.select((s.id, s.first_name, s.last_name) for s in db.Student)))
    # Need to get *all* the problems/graders and have them ordered, so that the index in this tuple can be mapped
    # to the id in the database.
    problems = tuple(orm.select(p.name for p in db.Problem).order_by('p.id'))
    graders = tuple(map(' '.join, orm.select((g.first_name, g.last_name) for g in db.Grader).order_by('g.id')))

## Mutable app state

# the solution that we are pointing to
submission_id = 1
problem_id = 1
grader_id = None
loaded_feedback = set()
loaded_remarks = ""

## Solution selection (submission x problem) matrix
prev_stud = Button(icon='fa-step-backward', description=' previous')
next_stud = Button(icon='fa-step-forward', description=' next')
next_ungraded_stud = Button(icon='fa-forward', description=' next ungraded')
jump_to_stud = Text(description='Jump to', placeholder='student # or name')


jump_to_problem = Dropdown(description='Jump to', placeholder='problem #', options=problems)

## Add feedback option
new_option = Text(placeholder='New feedback option')

## Solution and FeedbackOption
options_feedback = SelectMultiple(name='feedback')
remarks = Textarea(placeholder='Student-specific feedback')
answer = Image(format='png', width='100%')

## info
graded_label = HTML()
graded_label.template = '<h5>Graded by: {} @ {:%d/%m %H:%M}</h5>'.format

problem_progress = IntProgress(
    value=0,
    min=0,
    max=n_submissions,
    step=1,
    description='Progress:',
    bar_style='success',
    orientation='horizontal',
)

## grader selection
grader = Dropdown(description='Grader', placeholder='Grader name', options=(None,) + graders)
grader.value = None
widgets_to_disable = (new_option, options_feedback, remarks)
for w in widgets_to_disable:
    w.disabled = True

# Widget behaviour

In [ ]:
### Controllers  -- hooking views (widgets) up to models

## Utilities

def increment(val, max_val):
    return val + 1 if val < max_val else 1

def decrement(val, max_val):
    return val - 1 if val > 1 else max_val

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

def add_option(target, value):
    with target.hold_trait_notifications():
        target.options = target.options + [value]
    
def load_image(path):
    with open(path, 'rb') as f:
        return f.read()

# render the solution widgets
def render_solution():
    with orm.db_session:
        # this works because Solution has '(submission, problem)' as its PrimaryKey
        solution = db.Solution.get(submission=submission_id, problem=problem_id)
        student = solution.submission.student
        possible_feedback = list(orm.select(f.text for f in db.FeedbackOption
                                            if solution.problem in f.problems).order_by('f.id'))
        # get actual data back before we finish the database transaction
        selected_feedback = [f.text for f in solution.feedback]
        if solution.graded_by:
            graded_by = ' '.join((solution.graded_by.first_name, solution.graded_by.last_name))
        else:
            graded_by = 'some guy'
        graded_at = solution.graded_at
        num_graded = orm.count(s for s in db.Solution if s.problem.id == problem_id and s.graded_at is not None)

    # save app state
    global loaded_feedback, loaded_remarks
    loaded_feedback = set(selected_feedback)
    loaded_remarks = solution.remarks
    
    # update the UI state
    answer.value = load_image(solution.image_path)
    options_feedback.options = possible_feedback
    options_feedback.value = selected_feedback
    remarks.value = solution.remarks
    jump_to_stud.value = '{} ({}, {})'.format(student.id, student.first_name, student.last_name)
    
    problem_progress.value = num_graded
    if graded_at is None:
        graded_label.visible = False
    else: 
        graded_label.value = graded_label.template(graded_by, graded_at)
        graded_label.visible = True
    
# save the selected feedback
def save_feedback():
    if grader_id is None:
        return  # should not save if the grader is not selected
    selected_feedback = set(options_feedback.value)
    if selected_feedback == loaded_feedback and remarks.value == loaded_remarks:
        return  # nothing was changed -- do not save
    with orm.db_session:
        solution = db.Solution.get(submission=submission_id, problem=problem_id)
        solution.feedback =  list(orm.select(fb for fb in db.FeedbackOption if fb.text in selected_feedback))
        solution.remarks = remarks.value
        solution.graded_by = db.Grader[grader_id]
        solution.graded_at = datetime.datetime.now()

def save_then_render(callback):
    
    def _(sender):
        save_feedback()
        callback(sender)
        render_solution()
    
    return _

## actual controllers - abusing decorator syntax FTW!

@prev_stud.on_click
@save_then_render
def _(sender):
    global submission_id
    submission_id = decrement(submission_id, n_submissions)

@next_stud.on_click
@save_then_render
def _(sender):
    global submission_id
    submission_id = increment(submission_id, n_submissions)

@next_ungraded_stud.on_click
@save_then_render
def _(sender):
    global submission_id
    try:
        submission_id = next(iter(orm.select(s.submission.id for s in db.Solution
                                             if s.problem.id == problem_id and s.graded_at is None)))
    except StopIteration:
        pass  # when everything is graded

@observe(jump_to_problem, names="value")
@save_then_render
def _(state):
    # 'problems' global var is list of *all* problems ordered by problem ID
    global problem_id
    problem_id = problems.index(state['new']) + 1  # IDs in database start from 1
    
@observe(grader, names="value")
def _(state):
    global grader_id
    for w in widgets_to_disable:
        w.disabled = not bool(state['new'])
    try:
        grader_id = graders.index(state['new']) + 1  # IDs in database start from 1
    except ValueError:
        grader_id = None

@new_option.on_submit
def _(sender):
    # add to database
    with orm.db_session:
        value = sender.value.strip()
        if value:
            option = db.FeedbackOption.get(text=sender.value)
            problem = db.Problem[problem_id]
            if not option:    
                db.FeedbackOption(text=sender.value, problems=problem)
            else:
                option.problems += [problem]
            # update UI state manually rather than doing a full re-render
            add_option(options_feedback, value)
    sender.value = ''

@jump_to_stud.on_submit
@save_then_render
def _(sender):
    global submission_id
    try:
        student_id = int(sender.value.strip()[:7])
    except ValueError:
        return
    try:
        with orm.db_session:
            submission_id = db.Student.get(id=student_id).submission.id
    except AttributeError:
        return

# Layout

In [ ]:
render_solution()

In [ ]:
HBox([prev_stud, next_stud, next_ungraded_stud, jump_to_stud, jump_to_problem])

In [ ]:
HBox([VBox([options_feedback, new_option]), remarks])

In [ ]:
HBox([problem_progress, grader, graded_label])

In [ ]:
answer

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(jump_to_stud.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=students)))
display.display_javascript(prevent_unselect)